# Connections management

This section cosiders capabilities of the nginx to manage connections.

This section requires `http_stub_status_module` to be installed in nginx, so a custom build of nginx is used here.

In [16]:
docker run -itd --name experiment_nginx --rm -p 80:80 custom_nginx

a481aba9f5ef3bbcd4a7dab744bfc3bdcf5cc0402dff5e5d99a8038b0951ced3


**Note:** Don't forget to clean up the environment.

In [15]:
docker stop experiment_nginx

experiment_nginx


## Connections monitoring

You can check information about active connections using tools provided by `http_stub_status_module`. Check [corresponding page](https://nginx.org/en/docs/http/ngx_http_stub_status_module.html) of the official documentation.

---

The following cell creates an endpoint that uses the `stub_status' directive to create a special endpoint that provides information about the number of busy workers.

In [ ]:
docker exec -i experiment_nginx sh -c 'cat >  /etc/nginx/nginx.conf' << EOF
events {}
http {
    server {
        location = /basic_status {
            stub_status;
        }
    }
}
EOF

docker exec experiment_nginx nginx -s reload

Here is the result of invoking the built endpoint.

In [18]:
curl localhost:80/basic_status

Active connections: 1 
server accepts handled requests
 1 1 1 
Reading: 0 Writing: 1 Waiting: 0 


## Worker connections

Nginx provides a configuration option to specify how many simultaneous connections it can support. This is controlled by the `worker_connections` directive within the `events` block. It defines the maximum number of connections Nginx can handle at the same time. 

For more details, refer to the [official documentation](https://nginx.org/en/docs/ngx_core_module.html#worker_connections).

---

The following cell sets up nginx to have only two worker connections - that's the minimum number for nginx to run.

In [19]:
docker exec -i experiment_nginx sh -c 'cat >  /etc/nginx/nginx.conf' <<EOF
events {worker_connections 2;}
http {server{return 200 "hello";}}
EOF
docker exec -i experiment_nginx nginx -s reload

Any message will simply return nothing.

In [20]:
curl localhost:80

curl: (52) Empty reply from server


: 52

The same configuration with `worker_connections` set to 3 works fine.

In [21]:
docker exec -i experiment_nginx sh -c 'cat >  /etc/nginx/nginx.conf' <<EOF
events {worker_connections 3;}
http {server{return 200 "hello";}}
EOF
docker exec -i experiment_nginx nginx -s reload

In [22]:
curl localhost:80

hello
